In [2]:
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage
import gradio as gr
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os
from gtts import gTTS
import tempfile

In [3]:
# 채팅 기록을 포함하여 응답을 생성하는 함수
def chat(message, history):
    # ChatOllama 모델 초기화
    model = ChatOllama(model="gemma2", temperature=0.7, verbose=False)
    # 이전 대화 기록을 ChatOllama 형식으로 변환
    chat_history = []
    for human, ai in history:
        chat_history.append(HumanMessage(content=human))
        chat_history.append(AIMessage(content=ai))
        
    # 현재 메세지 추가
    chat_history.append(HumanMessage(content=message))
    
    # 모델을 사용하여 응답 생성
    response = model.invoke(chat_history)
    return response.content

In [4]:
def text_to_speech(text, lang = 'ko'):
    #임심 파일 생성
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as fp:
        temp_filename = fp.name
    
    # TTS 변환
    #gtts를 활용
    tts = gTTS(text = text, lang = lang)
    tts.save(temp_filename)
    return temp_filename

In [5]:
def process_tts(text, lang):
    if not text:
        return None, "텍스트를 입력해주세요."
    try:
        audio_file = text_to_speech(text, lang)
        return audio_file, "변환이 완료되었습니다. 아래에서 재생 또는 다운로드할 수 있습니다."
    except Exception as e:
        return None, f"오류가 발생했습니다: {str(e)}"

In [6]:
# 워드클라우드 생성 함수
def generate_wordcloud():
    # 파일에서 텍스트 읽기
    with open('./dataset/history.txt', 'r', encoding='utf-8') as file:
        text = file.read()

    # 워드클라우드 생성
    wordcloud = WordCloud(
        font_path='malgun',  # 한글 폰트 설정 (맑은 고딕)
        background_color='white',
        width=800,
        height=600,
        max_words=200,
        max_font_size=100,
        min_font_size=10,
        random_state=42
    ).generate(text)

    # 워드클라우드 이미지를 파일로 저장
    wordcloud_path = "wordcloud_output.png"
    wordcloud.to_file(wordcloud_path)

    # Gradio가 파일 경로를 출력할 수 있도록 반환
    return wordcloud_path


In [7]:
# Gradio Tabbed Interface
with gr.Blocks() as iface:
    # Tab for Visualization (Word Cloud)
    with gr.Tab("Text To Speech"):
        gr.Interface(
            fn = process_tts,
            inputs=[
                gr.Textbox(lines=5, label="텍스트 입력"),
                gr.Dropdown(choices=['ko', 'en', 'ja', 'zh-cn'], label="언어 선택", value='ko')
            ],
            outputs=[
                gr.Audio(label="생성된 오디오"),
                gr.Textbox(label="상태 메세지")
            ],
            title = "Text to Speach Converter",
            description="텍스트를 입력하면 MP3 파일로 변환합니다."
        )
    
    with gr.Tab("AI 챗봇"):
        gr.ChatInterface(
            fn=chat,
            examples=[
                "안녕하세요!",
                "인공지능에 대해 설명해주세요.",
                "파이썬의 장점은 무엇인가요?"
            ],
            title = "AI 챗봇",
            description="질문을 입력하면 AI가 답변합니다."
        )
    # Tab for Word Cloud
    with gr.Tab("워드클라우드"):
        gr.Interface(
            fn=generate_wordcloud,
            inputs=None,  # 사용자 입력 없음
            outputs=gr.Image(label="워드클라우드"),  # 워드클라우드 이미지를 출력
            title="워드클라우드 생성기",
            description="history.txt 파일의 내용을 기반으로 워드클라우드를 생성합니다."
        )

c:\AIProject\.venv\lib\site-packages\gradio\components\chatbot.py:248: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


In [8]:
# 서버 실행
iface.launch(server_port=7861, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


In [9]:
#iface.close()